In [ ]:
# import all the necessary modules
import numpy as np
import pandas as pd
import wikipedia as wp

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
df.drop(0,inplace= True)

In [4]:
df = df.rename(columns={0:'PostalCode', 1:'Borough' ,2:'Neighbourhood'})

df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [5]:
df = df.reset_index(drop=True)

In [6]:
for i in range(0,len(df)):
 if df['Borough'][i] == 'Not assigned':
  df.drop(i,inplace=True) 

In [7]:
final_dataframe = pd.DataFrame(df).reset_index(drop=True)

In [9]:
final_dataframe = final_dataframe.rename( columns ={0:'PostalCode' , 1:'Borough' ,2:'Neighbourhood'} )

In [12]:
location= pd.read_csv('Geospatial_Coordinates.csv')

In [13]:

location.rename(columns={'Postal Code' : 'PostalCode'} ,inplace=True)

In [14]:
final_dataframe = pd.merge(final_dataframe, location, on='PostalCode' ,how ='inner')

In [15]:
location.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
final_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [17]:
final_dataframe.drop_duplicates(subset=['PostalCode'] ,keep='first' ,inplace=True)
final_dataframe.drop( 'PostalCode',axis=1,inplace=True)

In [22]:
toronto = final_dataframe[final_dataframe["Borough"].str.contains('Toronto')]

In [24]:

toronto.head().reset_index(drop=True)

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Ryerson,43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,East Toronto,The Beaches,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [26]:
from geopy.geocoders import Nominatim
locator = Nominatim()
location = locator.geocode('Toronto')
toronto_latitude = location.latitude
toronto_longitude = location.longitude

C:\Users\Tejas Shanbhag\AppData\Local\Continuum\anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [31]:
import folium

toronto_map = folium.Map([toronto_latitude , toronto_longitude] , zoom_start=11)

for lat ,lon ,borough ,neighborhood in zip( toronto['Latitude'] ,toronto['Longitude'] ,toronto['Borough'],toronto['Neighbourhood'] ):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=7, popup=label, color='black', fill=True, fill_color='#ccc131', fill_opacity=0.5)\
    .add_to(toronto_map)

toronto_map

![Kernel & front-end diagram](images/data_7.PNG)

In [28]:
CLIENT_ID = 'WXA2H4LKJEKPHUYE0NTWJ5UJQWFY4QBVYQV1FTATYR0LQXV3' # your Foursquare ID
CLIENT_SECRET = 'M1CULVBJUMGVO4S5IGBEQHQGSNH0UJ2HS5SEURHPZH1CR0TJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXA2H4LKJEKPHUYE0NTWJ5UJQWFY4QBVYQV1FTATYR0LQXV3
CLIENT_SECRET:M1CULVBJUMGVO4S5IGBEQHQGSNH0UJ2HS5SEURHPZH1CR0TJ


In [29]:
import requests
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant in'formation for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
Dovercourt Village
Harbourfront East
Little Portugal
The Danforth West
Design Exchange
Brockton
The Beaches West
Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
High Park
North Toronto West
The Annex
Parkdale
Davisville
Harbord
Runnymede
Moore Park
Chinatown
Deer Park
CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
First Canadian Place
Church and Wellesley
Business reply mail Processing Centre969 Eastern


In [32]:
toronto_venues


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.654260,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
5,Harbourfront,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,Harbourfront,43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,Harbourfront,43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,Harbourfront,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
9,Harbourfront,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [33]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
toronto_onehot.shape

(1710, 237)

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01,0.0
1,Berczy Park,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0
2,Brockton,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0
3,Business reply mail Processing Centre969 Eastern,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0
4,CN Tower,0.00,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0


In [36]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4                  Gym  0.04
5  American Restaurant  0.04
6                  Bar  0.03
7           Restaurant  0.03
8                Hotel  0.03
9       Cosmetics Shop  0.02


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3          Restaurant  0.04
4              Bakery  0.04
5  Seafood Restaurant  0.04
6          Steakhouse  0.04
7            Beer Bar  0.04
8         Cheese Shop  0.04
9                Café  0.04


----Brockton----
                    venue  freq
0             Coffee Shop  0.14
1          Breakfast Spot  0.09
2                    Café  0.09
3           Burrito Place  0.05
4                 Stadium  0.05
5      Falafel Restaurant  0.05
6  Furniture / Home Store  0.05
7                     Bar  0.05
8    Caribbean Restaurant  

                         venue  freq
0                  Coffee Shop  0.14
1                    Gift Shop  0.14
2                    Piano Bar  0.07
3                      Dog Run  0.07
4                 Burger Joint  0.07
5                          Bar  0.07
6                   Restaurant  0.07
7  Eastern European Restaurant  0.07
8                Movie Theater  0.07
9                 Dessert Shop  0.07


----Rosedale----
                       venue  freq
0                       Park  0.50
1                 Playground  0.25
2                      Trail  0.25
3          Accessories Store  0.00
4                Music Venue  0.00
5   Mediterranean Restaurant  0.00
6                Men's Store  0.00
7              Metro Station  0.00
8         Mexican Restaurant  0.00
9  Middle Eastern Restaurant  0.00


----Roselawn----
                        venue  freq
0                Home Service   0.5
1                      Garden   0.5
2           Accessories Store   0.0
3                Neighborh

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Gym,Thai Restaurant,American Restaurant,Bar,Restaurant,Hotel,Japanese Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Farmers Market
2,Brockton,Coffee Shop,Café,Breakfast Spot,Grocery Store,Italian Restaurant,Performing Arts Venue,Pet Store,Office,Gym,Nightclub
3,Business reply mail Processing Centre969 Eastern,Comic Shop,Butcher,Skate Park,Spa,Farmers Market,Light Rail Station,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
4,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Plane
5,Cabbagetown,Coffee Shop,Restaurant,Café,Pizza Place,Chinese Restaurant,Bakery,Convenience Store,Italian Restaurant,Indian Restaurant,Pub
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Spa
7,Chinatown,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Hotpot Restaurant
8,Christie,Grocery Store,Café,Park,Convenience Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Pub,Gastropub,Café,Bubble Tea Shop


In [39]:
from sklearn.cluster import KMeans
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
manhattan_merged = toronto

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.head()

C:\Users\Tejas Shanbhag\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Mexican Restaurant,Theater,Spa,Bank
13,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Restaurant,Ramen Restaurant,Bookstore,Diner
27,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Park
36,East Toronto,The Beaches,43.676357,-79.293031,0,Music Venue,Dance Studio,Neighborhood,Pub,Coffee Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
37,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Farmers Market


In [44]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood'],toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

![Kernel & front-end diagram](images/data_8.PNG)

In [48]:
clusters = toronto

# add clustering labels
clusters['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
clusters = clusters.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

clusters.head().reset_index(drop=True)

C:\Users\Tejas Shanbhag\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Mexican Restaurant,Theater,Spa,Bank
1,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Restaurant,Ramen Restaurant,Bookstore,Diner
2,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Park
3,East Toronto,The Beaches,43.676357,-79.293031,0,Music Venue,Dance Studio,Neighborhood,Pub,Coffee Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Farmers Market


### The first cluster can be visualized as:

In [50]:
clusters.loc[clusters['Cluster Labels'] == 0, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,Coffee Shop,Bakery,Park,Café,Pub,Breakfast Spot,Mexican Restaurant,Theater,Spa,Bank
13,Ryerson,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Restaurant,Ramen Restaurant,Bookstore,Diner
27,St. James Town,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Park
36,The Beaches,Music Venue,Dance Studio,Neighborhood,Pub,Coffee Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
37,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Bakery,Farmers Market
41,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Japanese Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Bar,Spa
42,Christie,Grocery Store,Café,Park,Convenience Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Coffee Shop
49,Adelaide,Coffee Shop,Café,Steakhouse,Gym,Thai Restaurant,American Restaurant,Bar,Restaurant,Hotel,Japanese Restaurant
52,Dovercourt Village,Supermarket,Bakery,Pharmacy,Portuguese Restaurant,Music Venue,Discount Store,Café,Middle Eastern Restaurant,Brewery,Liquor Store
61,Harbourfront East,Coffee Shop,Hotel,Pizza Place,Aquarium,Café,Brewery,Scenic Lookout,Sports Bar,Italian Restaurant,Sushi Restaurant


### The second cluster can be visualized as

In [51]:

clusters.loc[clusters['Cluster Labels'] == 1, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
112,Lawrence Park,Bus Line,Park,Swim School,Dim Sum Restaurant,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
130,North Toronto West,Coffee Shop,Clothing Store,Sporting Goods Shop,Park,Shoe Store,Sandwich Place,Salon / Barbershop,Café,Chinese Restaurant,Dessert Shop
145,Runnymede,Coffee Shop,Café,Pizza Place,Italian Restaurant,Bookstore,Sushi Restaurant,Bar,Fish & Chips Shop,Indie Movie Theater,Pharmacy


### The Thirdcluster can be visualized as 

In [53]:
clusters.loc[clusters['Cluster Labels'] == 2, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
150,Moore Park,Gym,Playground,Summer Camp,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
